In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

from modelclass import model


# A very small model and some data 

In [12]:
fmodel      = '''
frml <> a = c(-1) + b  + 7* x**2 $ 
frml <> d1 = 4*x + 3 * e(-1)+ c **2 +a  $ 
frml <> d3 = x + 3 * d1*a(+1)+c **3 $  
Frml <> x = 0.5 *e**2+x(-1)  $'''
mmodel = model(fmodel,modelname = 'My first model')
df = pd.DataFrame({'B': [1,1,1,1],'C':[1,2,3,4],'E':[4,4,4,4]},index=[2018,2019,2020,2021])
base = mmodel(df)
base

,B,C,E,D1,A,D3,X
2018,1.0,1.0,4.0,0.0,0.0,0.0,0.0
2019,1.0,2.0,4.0,498.0,450.0,2681746.0,8.0
2020,1.0,3.0,4.0,1880.0,1795.0,43.0,16.0
2021,1.0,4.0,4.0,0.0,0.0,0.0,0.0


# Create two newton instances one with symbolic one with numerical differentiation 

In [13]:
from modelnewton import newton_diff
newton = newton_diff(mmodel,forcenum = 0 ) # symbolic differentiation if possible 
newton_num = newton_diff(mmodel,forcenum = 1)  # Numerical differensation 

# Show the models which generates the derivatives 

A model is generated to evaluate all relevant derivatives



All possible derivatives involviong **lags** are constructed like this : 
$\frac{\partial{var_t}}{\partial{lhs\_var_{t-lag}}}= expression$  are constructed as: 

`frml <> <var>__p__<lhs_var>___lag___<lag> = expression`
    
All possible derivatives involviong **leads** lags are constructed like this : 
$\frac{\partial{var_t}}{\partial{lhs\_var_{t+lead}}}= expression$  are constructed as: 

`frml <> <var>__p__<lhs_var>___lead___<lead> = expression`


In [14]:
print(newton.diff_model.equations)  # Symbolic differentiation 

FRML  <> A__p__X___lag___0 = 14*X   $
FRML  <> D1__p__A___lag___0 = 1   $
FRML  <> D1__p__X___lag___0 = 4   $
FRML  <> D3__p__A___lead___1 = 3*D1   $
FRML  <> D3__p__D1___lag___0 = 3*A(+1)   $
FRML  <> D3__p__X___lag___0 = 1   $
FRML  <> X__p__X___lag___1 = 1   $


In [5]:
print(newton_num.diff_model.equations) # Numerical differentiation 

FRML  <> A__p__X___lag___0 = ((C(-1)+B+7*(X+0.0025)**2)-(C(-1)+B+7*(X-0.0025)**2))/0.005   $
FRML  <> D1__p__A___lag___0 = ((4*X+3*E(-1)+C**2+(A+0.0025))-(4*X+3*E(-1)+C**2+(A-0.0025)))/0.005   $
FRML  <> D1__p__X___lag___0 = ((4*(X+0.0025)+3*E(-1)+C**2+A)-(4*(X-0.0025)+3*E(-1)+C**2+A))/0.005   $
FRML  <> D3__p__A___lag___1 = ((X+3*(A(-1)+0.0025)+C**3)-(X+3*(A(-1)-0.0025)+C**3))/0.005   $
FRML  <> D3__p__X___lag___0 = (((X+0.0025)+3*A(-1)+C**3)-((X-0.0025)+3*A(-1)+C**3))/0.005   $
FRML  <> X__p__X___lag___1 = ((0.5*E**2+(X(-1)+0.0025))-(0.5*E**2+(X(-1)-0.0025)))/0.005   $


In [6]:
stacked_df = newton.get_diff_df_tot()
stacked_df.applymap(lambda x:f'{x:,.2f}' if x != 0.0 else ' ')

per        2019                         2020                         2021  \
var           A     D1     D3       X      A     D1     D3       X      A   
per  var                                                                    
2019 A    -1.00                112.00                                       
     D1    1.00  -1.00           4.00                                       
     D3                 -1.00    1.00                                       
     X                          -1.00                                       
2020 A                                 -1.00                224.00          
     D1                                 1.00  -1.00           4.00          
     D3    3.00                                      -1.00    1.00          
     X                           1.00                        -1.00          
2021 A                                                              -1.00   
     D1                                                              1.00   
     D3                                 3.00                                
     X                                                        1.00          

per                             
var          D1     D3       X  
per  var                        
2019 A                          
     D1                         
     D3                         
     X                          
2020 A                          
     D1                         
     D3                         
     X                          
2021 A                  336.00  
     D1   -1.00           4.00  
     D3          -1.00    1.00  
     X                   -1.00

In [7]:
newton.show_diff_latex()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

|                                      |   2019 |   2020 |   2021 |
|:-------------------------------------|-------:|-------:|-------:|
| $\frac{\partial D1_t}{\partial A_t}$ |      1 |      1 |      1 |
| $\frac{\partial D1_t}{\partial X_t}$ |      4 |      4 |      4 |

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

|                                          |   2019 |   2020 |   2021 |
|:-----------------------------------------|-------:|-------:|-------:|
| $\frac{\partial D3_t}{\partial A_{t-1}}$ |      3 |      3 |      3 |
| $\frac{\partial D3_t}{\partial X_t}$     |      1 |      1 |      1 |

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

|                                         |   2019 |   2020 |   2021 |
|:----------------------------------------|-------:|-------:|-------:|
| $\frac{\partial X_t}{\partial X_{t-1}}$ |      1 |      1 |      1 |

In [8]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Differentiation .ipynb'])

0